In [1]:
import igraph as ig
import numpy as np
from statistics import mean

import sympy.combinatorics as combinatorics

import sage.groups.perm_gps.permgroup as sage_permgroup
import sage.groups.perm_gps.permgroup_named as sage_named_groups

#from functions import *

get_names_from_list_of_ids = lambda ig_graph, ids : [ig_graph.vs["name"][j] for j in ids]

class NormalSubgroup:
	def __init__(self, ig_graph, sector, permutations):
		self.ig_graph = ig_graph.copy()
		self.sector = sector
		self.sector_names = get_names_from_list_of_ids(self.ig_graph, sector)
		self.sympy_permutations = permutations
		self.sympy_permutation_subgroup = combinatorics.PermutationGroup(self.sympy_permutations)
		
		self.orbits = [orbit for orbit in self.sympy_permutation_subgroup.orbits() if len(orbit) > 1]
		self.orbits = [get_names_from_list_of_ids(ig_graph, list(orbit)) for orbit in self.orbits]
		
		self.classification = self.classify_sympy_perm_group(self.sympy_permutations, len(self.sector))

		#repairedEdgesBool = np.array(self.ig_graph.es["color"]) == "Repaired"
		#repairedEdgesBool = list(np.flatnonzero(repairedEdgesBool))
		originalGraph = ig_graph.copy()
		#originalGraph.delete_edges(repairedEdgesBool)
		self.original_adjacency_matrix = np.array(originalGraph.get_adjacency().data)
	
	def calculate_epsilon(self, permutation):
		permutation_array = permutation.array_form
		permutation_matrix = np.identity(len(permutation_array))[permutation_array,:]

		commutator = np.matmul(permutation_matrix, self.original_adjacency_matrix) - np.matmul(self.original_adjacency_matrix, permutation_matrix)

		epsilon = np.sum(np.absolute(commutator))
		return epsilon
		
	def calculate_epsilons(self):
		self.epsilons = []
		for permutation in self.sympy_permutations:
			self.epsilons.append(self.calculate_epsilon(permutation))

		self.meanEpsilon = mean(self.epsilons)
		self.maxEpsilon = max(self.epsilons)

	#############################################################################
	########################### Helping functions ###############################
	#############################################################################
	def sympy_permutations_to_sage_group(self, sympy_permutations):
		subgroup_perms_sage_form = []
		
		for i in range(len(sympy_permutations)):
			permutation = sympy_permutations[i].cyclic_form
			permutation_sage = []
			for j in range(len(permutation)):
				permutation_sage.append(tuple(permutation[j]))
			subgroup_perms_sage_form.append(permutation_sage)

		sage_group = sage_permgroup.PermutationGroup(subgroup_perms_sage_form)
		return(sage_group)

	def classify_sympy_perm_group(self, sympy_permutations, domain_size):
		sage_group = self.sympy_permutations_to_sage_group(sympy_permutations)

		group_name = "Unknown"
		for i in range(1, domain_size + 1):
			if(sage_group.is_isomorphic(sage_named_groups.SymmetricGroup(i))):
				group_name = "S" + str(i)
				return(group_name)
		
		for i in range(1, domain_size + 1):
			if(sage_group.is_isomorphic(sage_named_groups.DihedralGroup(i))):
				group_name = "D" + str(i)
				return(group_name)

		for i in range(1, domain_size + 1):
			if(sage_group.is_isomorphic(sage_named_groups.CyclicPermutationGroup(i))):
				group_name = "C" + str(i)
				return(group_name)

		for i in range(1, domain_size + 1):
			if(sage_group.is_isomorphic(sage_named_groups.AlternatingGroup(i))):
				group_name = "A" + str(i)
				return(group_name)
			
		return(group_name)

	#############################################################################
	################################### Print ###################################
	#############################################################################
	def Pprint(self):
		print("Class:\n\t", self.classification)
		print("Mean epsilon = \t", self.meanEpsilon)
		print("Max epsilon = \t", self.maxEpsilon)
		print("Sector:\n\t", self.sector_names)
		print("Orbits (", len(self.orbits), "):")
		for i in range(len(self.orbits)):
			print("\t", i ,":\t", self.orbits[i])
		
		print("Permutations:")
		for i in range(len(self.sympy_permutations)):
			permutation = self.sympy_permutations[i]
			print("\tEpsilon = ", self.epsilons[i], ":\t", [get_names_from_list_of_ids(self.ig_graph, nodes) for nodes in permutation.cyclic_form])
		#print(self.sympy_permutation_subgroup)

	def print_info_to_file(self, filename):
		with open(filename, 'a') as filehandle:
			filehandle.writelines("Class: %s\n" % self.classification)
			filehandle.writelines("Mean epsilon = %s\n" % self.meanEpsilon)
			filehandle.writelines("Max epsilon = %s\n" % self.maxEpsilon)
			filehandle.writelines("Sector:\n\t%s\n" % self.sector_names)
			
			filehandle.writelines("Orbits (%i):\n" % len(self.orbits))
			for i in range(len(self.orbits)):
				filehandle.writelines("\t%i:\t%s\n" % (i, self.orbits[i]))
			
			filehandle.writelines("Permutations:\n")
			for i in range(len(self.sympy_permutations)):
				permutation = self.sympy_permutations[i]
				filehandle.writelines("\tEpsilon = %s:\t%s\n" % (self.epsilons[i], [get_names_from_list_of_ids(self.ig_graph, nodes) for nodes in permutation.cyclic_form]))
			filehandle.writelines("\n")
	
	def print_raw_permutations_to_file(self, filename):
		with open(filename, 'a') as filehandle:
			for j in range(len(self.sympy_permutations)):
				filehandle.writelines("%s\n" % self.sympy_permutations[j].array_form)
			filehandle.writelines("\n")

In [2]:
import igraph as ig
import pynauty as pynauty
import pandas as pd
import sys as sys
from collections import defaultdict

class SymmetryAnalyzer:
	def __init__(self, edges, directed = False, simplify = False):
		self.edges = edges
		
		self.ig_graph = ig.Graph.Read_Ncol(self.edges, directed=True, names=True)
		if (simplify==True):
			self.ig_graph = self.ig_graph.simplify(multiple=True)
		#self.ig_graph = ig.Graph.TupleList(self.edges[["Source", "Target", "Color"]].itertuples(index = False),
		#								   directed = directed, edge_attrs = "color")
		
		self.pynauty_graph = self.get_pynauty_graph_from_ig_graph(self.ig_graph)
		
		self.generators, grpsize1, grpsize2, orbits, numorbits = pynauty.autgrp(self.pynauty_graph)
		if (len(self.generators) == 0):
				print("Graph has no automorphisms")
				sys.exit()

		self.permutations = [combinatorics.Permutation(generator) for generator in self.generators]
		
		self.nodes = pd.DataFrame({"Id": self.ig_graph.vs["name"],
                                   "Label": self.ig_graph.vs["name"],
								   "Orbit": orbits})
	
	def decompose(self):
		self.sectors = self.get_sectors_from_permutations(self.permutations)
		self.nodes["Sector"] = self.get_assigned_sectors(self.sectors, self.ig_graph)
		self.nodes["Orbit"] = self.nodes["Sector"].astype(str) + "_" + self.nodes["Orbit"].astype(str)
		
		self.NormalSubgroups = [NormalSubgroup(self.ig_graph, self.sectors[i],
											   self.get_permutations_on_sector_id(self.permutations,
																				  self.sectors, i))
								for i in range(len(self.sectors))]
		
		self.calculate_epsilons()
	
	def calculate_epsilons(self):
		for NormalSubgroup in self.NormalSubgroups:
			NormalSubgroup.calculate_epsilons()
		
		meanEpsilons = [NormalSubgroup.meanEpsilon for NormalSubgroup in self.NormalSubgroups]
		maxEpsilons = [NormalSubgroup.maxEpsilon for NormalSubgroup in self.NormalSubgroups]
		self.meanEpsilon = mean(meanEpsilons)
		self.maxEpsilon = max(maxEpsilons)
	
	# List of indices (update it every time you change anything as it's used twice in the print info section):
	# Fiedler value
	# Normalized Fiedler value
	# Eigen ratio
	###### experimental indices:
	# Random Walk Fiedler Value
	# Minimal degree
	# Maximal degree
	# Sum of degrees
	# Vertex connectivity
	# Edge connectivity
	# Randic Index
	# Average path length
	# Average clustering coefficient (local)
	def calculate_indices(self):
		self.calculate_fiedler_values()
		self.calculate_topological_indices()
		self.calculate_experimental_values()
	
	#############################################################################
	########################### Helping functions ###############################
	#############################################################################
	def get_pynauty_graph_from_ig_graph(self, ig_graph):
		# Represent igraph as adjacency list-of-dictionaries
		adjacency_dict = {}
		for source in ig_graph.vs.indices:
			adjacent_edges = ig_graph.es.select(_source = source)
			adjacent_nodes = []
			for edge in adjacent_edges:
				if(edge.source == source):
					adjacent_nodes.append(edge.target)
				else:
					adjacent_nodes.append(edge.source)
			adjacency_dict[source] = adjacent_nodes

		pynauty_graph = pynauty.Graph(number_of_vertices = ig_graph.vcount(),
									  directed = ig_graph.is_directed(),
									  adjacency_dict = adjacency_dict)
		
		return(pynauty_graph)

	def get_sectors_from_permutations(self, permutations):
		permutation_domains = []
		for permutation in permutations:
			permutation_domains.append([value for sublist in permutation.cyclic_form for value in sublist])
			
		return(list(self.merge_common(permutation_domains)))

	# this is the function taken from the internet, it would be better to re-write it
	def merge_common(self, lists):
		neigh = defaultdict(set)
		visited = set()
		for each in lists:
			for item in each:
				neigh[item].update(each)
		def comp(node, neigh = neigh, visited = visited, vis = visited.add):
			nodes = set([node])
			next_node = nodes.pop
			while nodes:
				node = next_node()
				vis(node)
				nodes |= neigh[node] - visited
				yield node
		for node in neigh:
			if node not in visited:
				yield sorted(comp(node))

	def get_permutations_on_sector_id(self, permutations, sectors, sector_id):
		is_permutation_on_sector = lambda permutation, sector_id : all([node in set(sectors[sector_id]) for node in permutation])

		permutations_to_return = []
		for permutation in permutations:
			permutation_cyclic = [value for sublist in permutation.cyclic_form for value in sublist]
			if(is_permutation_on_sector(permutation_cyclic, sector_id)):
				permutations_to_return.append(permutation)
		return(permutations_to_return)
		
	def get_assigned_sectors(self, sectors, ig_graph):
		sectorId = []
		for i in ig_graph.vs.indices:
			search_result = [i in sector for sector in sectors]
			if (any(search_result) == False):
				sectorId.append(-1)
			else:
				sectorId.append([idx + 1 for idx in range(len(search_result)) if search_result[idx] == True][0])
		
		trivialIdx = max(sectorId) + 1
		for idx in range(len(sectorId)):
			if sectorId[idx] == -1:
				sectorId[idx] = trivialIdx
				trivialIdx = trivialIdx + 1
		return(sectorId)
	#############################################################################
	########################## Calculating indices ##############################
	#############################################################################
	def calculate_fiedler_values(self):
		graph_laplacian = self.ig_graph.laplacian(normalized = False)
		graph_laplacian = np.array(graph_laplacian)
		eigenval, eigenvect = np.linalg.eig(graph_laplacian)
		# we only take the real part, because imaginary part is very small and
		# it seems like it's here due to a computational artifact
		eigenval = eigenval.real
		eigenval = np.sort(eigenval)
		self.fiedlerValue = eigenval[1]
		self.eigenRatio = eigenval[1] / eigenval[len(eigenval) - 1]
		
		graph_laplacian = self.ig_graph.laplacian(normalized = True)
		graph_laplacian = np.array(graph_laplacian)
		eigenval, eigenvect = np.linalg.eig(graph_laplacian)
		# we only take the real part, because imaginary part is very small and
		# it seems like it's here due to a computational artifact
		eigenval = eigenval.real
		eigenval = np.sort(eigenval)
		self.normalizedFiedlerValue = eigenval[1]
	
	def calculate_experimental_values(self):
		adjacency_matrix = np.array(self.ig_graph.get_adjacency().data)
		eigenval, eigenvect = np.linalg.eig(adjacency_matrix)
		eigenval = np.sort(eigenval)
		kappa = eigenval[len(eigenval) - 1] + 5
		
		graph_laplacian = self.ig_graph.laplacian(normalized = False)
		graph_laplacian = np.array(graph_laplacian)
		degreeSeq = np.array(self.ig_graph.degree())
		randomWalkLaplacian = graph_laplacian / degreeSeq[:, None]
		eigenval, eigenvect = np.linalg.eig(randomWalkLaplacian)
		# we only take the real part, because imaginary part is very small and
		# it seems like it's here due to a computational artifact
		eigenval = eigenval.real
		eigenval = np.sort(eigenval)
		self.randomWalkSecondEigenvalue = eigenval[1]
		
		degreeSeq = np.sort(np.array(self.ig_graph.degree()))
		self.minimalDegree = np.ndarray.min(degreeSeq)
		self.maximalDegree = np.ndarray.max(degreeSeq)
		self.sumDegrees = np.sum(degreeSeq)
		
		self.vertex_connectivity = self.ig_graph.vertex_connectivity()
		self.edge_connectivity = self.ig_graph.edge_connectivity()

	def calculate_topological_indices(self):
		# Randic Index
		degrees = self.ig_graph.degree()
		randic_index = []
		for i in range(self.ig_graph.vcount()):
			for j in range(self.ig_graph.vcount()):
				if(not self.ig_graph.are_connected(i, j)):
						randic_index.append(1 / (degrees[i] * degrees[j])**(1/2))
		self.randic_index = sum(randic_index)

		# Average path length
		self.average_path_length = self.ig_graph.average_path_length()

		# Average clustering coefficient (local)
		self.average_clustering_coefficient = self.ig_graph.transitivity_avglocal_undirected(mode = 'zero')

	#############################################################################
	################################### Print ###################################
	#############################################################################
	def print_all_info(self):
		nodes = self.nodes.sort_values(by = ["Sector", "Orbit"])
		print(self.nodes)
		
		self.print_indices()
		print()
		self.print_normal_subgroups()
	
	def print_all_info_to_files(self, prefix):
		self.print_indices_to_file(prefix + "_indices.txt")
		self.print_normal_subgroups_to_file(prefix + "_normal_subgroups.txt")
		self.print_permutations_to_file(prefix + "_permutations.txt")
		self.print_nodes_to_file(prefix + "_nodes.csv")
		self.print_edges_to_file(prefix + "_edges.csv")
	
	def print_normal_subgroups(self):
		for NormalSubgroup in self.NormalSubgroups:
			NormalSubgroup.print()
			print()
	
	def print_normal_subgroups_to_file(self, fileName):
		with open(fileName, 'w') as filehandle:
			pass
		for NormalSubgroup in self.NormalSubgroups:
			NormalSubgroup.print_info_to_file(fileName)

	def print_permutations_to_file(self, fileName):
		with open(fileName, 'w') as filehandle:
			filehandle.writelines("%s\n" % self.ig_graph.vs["name"])
		for NormalSubgroup in self.NormalSubgroups:
			NormalSubgroup.print_raw_permutations_to_file(fileName)
	
	def print_nodes_to_file(self, fileName):
		nodes = self.nodes.sort_values(by = ["Sector", "Orbit"])
		nodes.to_csv(fileName, index = False)
	
	def print_edges_to_file(self, fileName):
		edges = self.edges
		edges["Type"] = "Undirected"
		edges.to_csv(fileName, index = False)
	
	def print_indices(self):
		print("Fiedler value = ", self.fiedlerValue)
		print("Normalized Fiedler value = ", self.normalizedFiedlerValue)
		print("Eigen ratio = ", self.eigenRatio)
		print("Mean epsilon = ", self.meanEpsilon)
		print("Max epsilon = ", self.maxEpsilon)
		print("Randic Index = ", self.randic_index)
		print("Average Path Length = ", self.average_path_length)
		print("Average Clustering Coefficient = ", self.average_clustering_coefficient)
	
	def print_indices_to_file(self, fileName):
		with open(fileName, 'w') as filehandle:
			filehandle.writelines("Fiedler value,%s\n" % self.fiedlerValue)
			filehandle.writelines("Normalized Fiedler Value,%s\n" % self.normalizedFiedlerValue)
			filehandle.writelines("Eigen ratio,%s\n" % self.eigenRatio)
			filehandle.writelines("Mean epsilon,%s\n" % self.meanEpsilon)
			filehandle.writelines("Max epsilon,%s\n" % self.maxEpsilon)
			# experimental indices from here down
			filehandle.writelines("Random Walk Fiedler Value,%s\n" % self.randomWalkSecondEigenvalue)
			filehandle.writelines("Minimal degree,%s\n" % self.minimalDegree)
			filehandle.writelines("Maximal degree,%s\n" % self.maximalDegree)
			filehandle.writelines("Sum of degrees,%s\n" % self.sumDegrees)
			filehandle.writelines("Vertex connectivity,%s\n" % self.vertex_connectivity)
			filehandle.writelines("Edge connectivity,%s\n" % self.edge_connectivity)
			filehandle.writelines("Randic Index,%s\n" % self.randic_index)
			filehandle.writelines("Average Path Length,%s\n" % self.average_path_length)
			filehandle.writelines("Average Clustering Coefficient,%s\n" % self.average_clustering_coefficient)

In [4]:
DATA = SymmetryAnalyzer('BCVWLRHS_no_header.tsv',directed=True, simplify=True)

In [5]:
filename = 'BCVWLRHS_no_header'
f= open(filename+'.txt',"w")
DATA = SymmetryAnalyzer(filename+'.tsv',directed=True, simplify=True)
GRAPH = DATA.ig_graph
SECTORS = DATA.get_sectors_from_permutations(DATA.permutations)

for i in range(len(SECTORS)):
    sec = SECTORS[i]
    pergroup = NormalSubgroup(GRAPH, SECTORS[i], DATA.get_permutations_on_sector_id(DATA.permutations,SECTORS, i))
    Type = pergroup.classify_sympy_perm_group(pergroup.sympy_permutations,len(SECTORS))
    for j in range(len(SECTORS[i])):
        sec[j] = GRAPH.vs[sec[j]]['name']
    print('Support '+str(sec)+' has permutation symmetry of the '+str(Type)+' type.\n')
    f.write('Support '+str(sec)+' has permutation symmetry of the '+str(Type)+' type.\n')
f.close()

Support ['AIBL', 'AVAL', 'AVEL', 'AVER', 'AIBR', 'AVAR', 'AVDL', 'AVDR'] has permutation symmetry of the S2 type.

Support ['RIML', 'RIMR'] has permutation symmetry of the S2 type.

Support ['VA08', 'VA09'] has permutation symmetry of the S2 type.

Support ['VA02', 'VA03', 'VA04', 'VA05'] has permutation symmetry of the S2 type.

Support ['DA06', 'DA07', 'VA10'] has permutation symmetry of the S3 type.

Support ['DA01', 'DA02', 'DA03', 'DA04'] has permutation symmetry of the S4 type.

Support ['DA05', 'DA08', 'DA09', 'VA06', 'VA11'] has permutation symmetry of the S5 type.



In [6]:
print(SECTORS)

[['AIBL', 'AVAL', 'AVEL', 'AVER', 'AIBR', 'AVAR', 'AVDL', 'AVDR'], ['RIML', 'RIMR'], ['VA08', 'VA09'], ['VA02', 'VA03', 'VA04', 'VA05'], ['DA06', 'DA07', 'VA10'], ['DA01', 'DA02', 'DA03', 'DA04'], ['DA05', 'DA08', 'DA09', 'VA06', 'VA11']]


In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [9]:
DATA.nodes

,Id,Label,Orbit
0,AIBL,AIBL,0
1,AVAL,AVAL,1
2,AVEL,AVEL,2
3,AVER,AVER,2
4,RIML,RIML,4
5,RIMR,RIMR,4
6,AIBR,AIBR,0
7,AVAR,AVAR,1
8,AVDL,AVDL,8
9,AVDR,AVDR,8


In [18]:
DATA.nodes.to_csv('orbits_extended.csv',sep='\t')